# S&P 500 Return Rate for the Past 5 Years 

## CMSC320 Final Tutorial Project
## By: Eric Qian, Sahil Sinha, Suraj Narang
## 2 Dec 2021

## Introduction

In this project we aim to demonstrate our learning and understanding of the data lifecycle by applying it to reality. Our primary goal in this tutorial is to try and determine whether it is a valid decision to invest in the Standard and Poor's 500 stock market index. 

### Background Info

This index essentially tracks the performance of 500 big companies that are enlisted on US stock exchanges. Often utilized as a benchmark for the whole US equity market, the S&P 500 index is prominently known to be one of the most frequently followed market indices. Rather than seeking to outperform the index through picking stocks, timing the market, or trading actively, these funds depend mainly on the diversification to build returns. The real question is that comes into mind is whether now is a great time to invest in the S&P 500.

### Used Libraries

- Pandas: Organized and portrayed the data within dataframes
- Seaborn: Helped construct plots to depict trends
* Numpy: Helped support the stock data
* matplotlib: Formatted all the plots effectively

## Data Loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel('Price of S&P500 on the First Day of Each Month.xlsx')
df.head(12)

,Year,Month,Stock Price ($)
0,January,2019,2510.03
1,February,2019,2706.53
2,March,2019,2803.69
3,April,2019,2867.19
4,May,2019,2923.73
5,June,2019,2744.45
6,July,2019,2964.33
7,August,2019,2953.56
8,September,2019,2906.27
9,October,2019,2940.25


The table above represents the monthly stock prices in the year of 2019.

In [2]:
df[12:24]

,Year,Month,Stock Price ($)
12,January,2020,3257.85
13,February,2020,3248.92
14,March,2020,3090.23
15,April,2020,2470.50
16,May,2020,2830.71
17,June,2020,3055.73
18,July,2020,3115.86
19,August,2020,3294.61
20,September,2020,3526.65
21,October,2020,3380.80


The table above represents the monthly stock prices in the year of 2020.

In [3]:
df[24:36]

,Year,Month,Stock Price ($)
24,January,2021,3700.65
25,February,2021,3773.86
26,March,2021,3901.82
27,April,2021,4019.87
28,May,2021,4192.66
29,June,2021,4202.04
30,July,2021,4319.94
31,August,2021,4387.16
32,September,2021,4524.09
33,October,2021,4357.04


The table above represents the monthly stock prices in the year of 2021. Each row would be distinguished by the date (year and month), as well as the stock price values. There are a total of 36 entries from the last two years.

## Data Cleaning

## Exploratory Data Analysis